In [17]:
import pandas as pd
import pickle

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Reading in our data

In [18]:
df = pd.read_csv("../data/50k_positional_reinf.csv", index_col=False)
df = df.drop_duplicates(subset=[str(x) for x in range(1, 10)])
df

,1,2,3,4,5,6,7,8,9,label,outcome,first
0,0,0,0,0,0,0,0,-1,0,4,-1,-1
1,0,-1,0,1,0,-1,1,1,-1,1,1,1
2,0,0,0,0,0,-1,0,0,0,4,-1,-1
3,0,0,0,0,0,0,0,0,0,4,1,1
4,0,0,0,0,-1,0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
79876,0,0,-1,1,1,-1,1,-1,0,9,1,1
86313,-1,1,-1,1,0,-1,1,0,0,9,1,1
86467,0,1,-1,-1,0,1,-1,0,1,8,-1,1
87483,0,1,1,-1,-1,0,-1,0,1,1,1,1


Splitting data into features and labels, then swapping to vectorized format.

In [19]:
cols = [str(x) for x in range(1, 10)]
cols += ["first"]
cols += ["outcome"]
X = df[cols]
y = df["label"]
X, y

(       1  2  3  4  5  6  7  8  9  first  outcome
 0      0  0  0  0  0  0  0 -1  0     -1       -1
 1      0 -1  0  1  0 -1  1  1 -1      1        1
 2      0  0  0  0  0 -1  0  0  0     -1       -1
 3      0  0  0  0  0  0  0  0  0      1        1
 4      0  0  0  0 -1  0  1  0  0      1        1
 ...   .. .. .. .. .. .. .. .. ..    ...      ...
 79876  0  0 -1  1  1 -1  1 -1  0      1        1
 86313 -1  1 -1  1  0 -1  1  0  0      1        1
 86467  0  1 -1 -1  0  1 -1  0  1      1       -1
 87483  0  1  1 -1 -1  0 -1  0  1      1        1
 93764  1  1 -1 -1  0 -1  1  0  0      1       -1
 
 [4520 rows x 11 columns],
 0        4
 1        1
 2        4
 3        4
 4        1
         ..
 79876    9
 86313    9
 86467    8
 87483    1
 93764    8
 Name: label, Length: 4520, dtype: int64)

In [20]:
X = X.values
y = y.values

X, y

(array([[ 0,  0,  0, ...,  0, -1, -1],
        [ 0, -1,  0, ..., -1,  1,  1],
        [ 0,  0,  0, ...,  0, -1, -1],
        ...,
        [ 0,  1, -1, ...,  1,  1, -1],
        [ 0,  1,  1, ...,  1,  1,  1],
        [ 1,  1, -1, ...,  0,  1, -1]], dtype=int64),
 array([4, 1, 4, ..., 8, 1, 8], dtype=int64))

Splitting training and validation data.

In [21]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [22]:
model = MLPClassifier(hidden_layer_sizes=(1000, 800, 600, 400, 200, 100, 50, 25), activation='relu', max_iter=200, learning_rate="adaptive")
model = model.fit(train_X, train_y)

In [23]:
train_preds = model.predict(train_X)
predictions = model.predict(val_X)
accuracy_score(predictions, val_y), accuracy_score(train_preds, train_y)

(0.41238938053097346, 1.0)

In [24]:
pickle.dump(model, open("../model_binaries/agent_ordered_adaptive_DEEP_relu.sav", "wb"))